In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

"""paper_pipeline_pseudocode.py
--------------------------------
Companion pseudo‑code for the manuscript:
“Predicting Antibiotic Resistance in *Neisseria gonorrhoeae* Clinical Isolates
 Using Machine‑ and Deep‑Learning”

This file is NOT meant to be executed as‑is; it captures the logical flow,
key algorithmic choices, and hyper‑parameters described in the paper in a
readable Pythonic format for reviewers and readers.

⚑  Replace each `TODO` section with concrete implementation details if you
   intend to run the pipeline.
"""

# ───────────────────────────────────────────────────────────────────────────────
# 0. Imports & constants
# ───────────────────────────────────────────────────────────────────────────────
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, matthews_corrcoef
)
from lazypredict.Supervised import LazyClassifier
from catboost import CatBoostClassifier
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
import shap  # SHAP explainability
# DeLong: see https://github.com/Netflix/vmaf/blob/master/python/setup.py or
# external libraries such as `deltapy`
# TODO: import DeLong test implementation (or custom function)

RNG_SEED = 42
N_FOLDS = 5
DATA_CSV = Path("metadata.csv")

# ───────────────────────────────────────────────────────────────────────────────
# 1. Load & basic EDA
# ───────────────────────────────────────────────────────────────────────────────
def load_data(csv_path: Path) -> pd.DataFrame:
    """Load the surveillance metadata."""
    df = pd.read_csv(csv_path)
    # TODO: perform sanity checks (shape, dtypes)
    return df


def basic_eda(df: pd.DataFrame) -> None:
    """Lightweight EDA: value counts, missingness, skew, etc."""
    # TODO: summarise distributions, skewness, and visualize if needed
    pass


# ───────────────────────────────────────────────────────────────────────────────
# 2. Pre‑processing
# ───────────────────────────────────────────────────────────────────────────────
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """Impute missing values and label‑encode categorical columns."""
    # example strategy from the paper:
    for col in df.columns:
        if df[col].dtype.kind in "biufc":  # numeric
            skew = df[col].skew()
            if skew > 1:
                df[col] = df[col].fillna(df[col].mean())
            elif skew < -1:
                df[col] = df[col].fillna(df[col].median())
            else:
                df[col] = df[col].fillna(df[col].median())
        else:  # categorical
            df[col] = df[col].fillna(df[col].mode()[0])
            df[col] = df[col].astype("category").cat.codes
    return df


# ───────────────────────────────────────────────────────────────────────────────
# 3. Feature / target split helper
# ───────────────────────────────────────────────────────────────────────────────
ANTIBIOTICS = {"azm_sr": "Azithromycin",
               "cip_sr": "Ciprofloxacin",
               "cfx_sr": "Cefixime"}


def make_xy(df: pd.DataFrame, target_key: str):
    """Return X, y numpy arrays for a given antibiotic resistance column."""
    y = df[target_key].values
    X = df.drop(columns=list(ANTIBIOTICS.keys())).values
    return X, y


# ───────────────────────────────────────────────────────────────────────────────
# 4. Baseline benchmarking using LazyPredict
# ───────────────────────────────────────────────────────────────────────────────
def benchmark_models(X, y):
    """Run 32 default classifiers to establish a performance floor."""
    clf = LazyClassifier(verbose=0, random_state=RNG_SEED, predictions=False)
    models, _ = clf.fit(X, X, y, y)  # train==test for quick listing
    return models.sort_values("Accuracy", ascending=False)


# ───────────────────────────────────────────────────────────────────────────────
# 5. CatBoost with Bayesian optimisation
# ───────────────────────────────────────────────────────────────────────────────
def train_catboost(X, y):
    """Tune CatBoost hyper‑parameters via Bayesian optimisation."""
    # Hyper‑parameter search space
    param_bounds = {
        "depth": (4, 10),
        "learning_rate": (0.005, 0.3),
        "l2_leaf_reg": (1, 10),
    }
    # TODO: use scikit‑optimize / optuna to iterate and maximise AUC
    best_params = {"depth": 6, "learning_rate": 0.05, "l2_leaf_reg": 3}
    model = CatBoostClassifier(
        **best_params,
        loss_function="Logloss",
        eval_metric="AUC",
        random_seed=RNG_SEED,
        verbose=False
    )
    model.fit(X, y)
    return model


# ───────────────────────────────────────────────────────────────────────────────
# 6. Keras feed‑forward neural network
# ───────────────────────────────────────────────────────────────────────────────
def build_ffnn(input_dim: int) -> Sequential:
    """Return a compiled 3‑layer feed‑forward network."""
    model = Sequential([
        Dense(256, activation="relu", input_shape=(input_dim,)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation="sigmoid"),
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss="binary_crossentropy",
                  metrics=["AUC", "accuracy"])
    return model


# ───────────────────────────────────────────────────────────────────────────────
# 7. Cross‑validation evaluation
# ───────────────────────────────────────────────────────────────────────────────
def cv_evaluate(model_fn, X, y, model_type="catboost"):
    """Stratified K‑fold CV returning key metrics."""
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RNG_SEED)
    metrics = {"AUC": [], "F1": [], "MCC": []}

    for train_idx, test_idx in skf.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        if model_type == "ffnn":
            model = model_fn(X_train.shape[1])
            model.fit(X_train, y_train, epochs=50, batch_size=32,
                      verbose=0, validation_split=0.1)
            y_prob = model.predict(X_test).ravel()
        else:  # catboost or any scikit‑compatible model
            model = model_fn(X_train, y_train)
            y_prob = model.predict_proba(X_test)[:, 1]

        y_pred = (y_prob >= 0.5).astype(int)
        metrics["AUC"].append(roc_auc_score(y_test, y_prob))
        metrics["F1"].append(f1_score(y_test, y_pred))
        metrics["MCC"].append(matthews_corrcoef(y_test, y_pred))

    return {k: np.mean(v) for k, v in metrics.items()}


# ───────────────────────────────────────────────────────────────────────────────
# 8. Statistical comparison (DeLong)
# ───────────────────────────────────────────────────────────────────────────────
def delong_test(preds1, preds2, y_true):
    """Return p‑value comparing two ROC AUCs (placeholder)."""
    # TODO: insert DeLong implementation or use `scikit‑posthocs`
    raise NotImplementedError


# ───────────────────────────────────────────────────────────────────────────────
# 9. SHAP explainability
# ───────────────────────────────────────────────────────────────────────────────
def explain_with_shap(model, X):
    """Compute SHAP values and return summary DataFrame."""
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    shap.summary_plot(shap_values, X, show=False)
    # TODO: save figure or return per‑feature mean(|SHAP|)
    pass


# ───────────────────────────────────────────────────────────────────────────────
# 10. Orchestration
# ───────────────────────────────────────────────────────────────────────────────
def main():
    df = load_data(DATA_CSV)
    basic_eda(df)

    df_clean = preprocess(df)

    for target in ANTIBIOTICS.keys():
        X, y = make_xy(df_clean, target)

        # Baseline benchmarks
        baselines = benchmark_models(X, y)
        print(f"\nTop‑5 baseline models for {target}:\n", baselines.head())

        # CatBoost
        cat_metrics = cv_evaluate(train_catboost, X, y, model_type="catboost")
        print(f"CatBoost CV metrics for {target}:", cat_metrics)

        # Neural net
        nn_metrics = cv_evaluate(build_ffnn, X, y, model_type="ffnn")
        print(f"Neural Net CV metrics for {target}:", nn_metrics)

        # TODO: external validation subset evaluation
        # TODO: DeLong tests comparing CatBoost vs literature

        # Explainability
        cat_model = train_catboost(X, y)
        explain_with_shap(cat_model, X)

    print("Pipeline complete.")


if __name__ == "__main__":
    main()
